In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/clean_total.csv")

In [4]:
df.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,arresting_agency,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment
0,679537,301831.0,1974-04-02,MALE,WHITE,M,SHAWNEE,KS,66226.0,15CR00773,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-04-01,2015-04-01,CA,10000.0,1,"MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MOD...",OTHERS,1
1,431835,296410.0,1971-01-01,MALE,WHITE,missing,OLATHE,KS,66062.0,13CR00129,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-01-19,2013-01-22,SUR,10000.0,1,SOCIAL PHOBIA,OTHERS,1
2,431835,296410.0,1971-01-01,MALE,WHITE,missing,OLATHE,KS,66062.0,14CR00932,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2014-05-02,2014-05-02,SUR,5000.0,1,SOCIAL PHOBIA,OTHERS,1
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021.0,15CR02335,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2015-10-07,2015-10-07,SUR,10000.0,1,NaN,NaN,0
4,655865,370683.0,1984-08-03,MALE,WHITE,S,OLATHE,KS,66062.0,13CR00468,...,JOHNSON COUNTY SHERIFF'S DEPARTMENT,COUNTY SHERIFF,2013-03-03,2013-03-03,SUR,1000.0,1,NaN,NaN,0


In [5]:
df['re_entry'] = df['dedupe_id'].map(df['dedupe_id'].value_counts())

In [7]:
df['re_entry'] = np.where(df['re_entry']>1, 1, 0)

In [9]:
cd code

/Users/jeeyoon_song/Desktop/syntaxerror-mlpp2018/code


In [12]:
import final_temporal

In [10]:
test = df.copy()

In [13]:
final_temporal.within_frame(test, col_name='re_entry')

ValueError: 

In [27]:
test = test[test['re_entry']==1]

In [32]:
test['booking_date'] = pd.to_datetime(test['booking_date'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [37]:
test['release_date'] = pd.to_datetime(test['release_date'])

In [14]:
def within_frame(df, id_num='dedupe_id', timestamp='booking_date', col_name='re-entry', duration=365):
	df = df.sort_values(by=[id_num, timestamp])
	df['{}'.format(col_name)] = df.groupby(id_num)[timestamp].diff()
	df['{}'.format(col_name)] = df['{}'.format(col_name)].apply(lambda x: x.days)
	df['within_{}'.format(duration)] = np.where(df['{}'.format(col_name)]>duration, 1, 0)    
	
	return df

In [33]:
test = test.sort_values(by=['dedupe_id', 'booking_date'])

In [34]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,arresting_agency_type,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry
20701,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR01499,...,CITY,2011-07-05,2011-07-05,PR,2500.0,1,NaN,NaN,0,1
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,COUNTY SHERIFF,2011-11-20,2012-10-22,0 SUR\ndtype: object,2500.0,0,NaN,NaN,0,1
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,CITY,2014-05-02,2014-05-19,SUR,1500.0,1,NaN,NaN,0,1
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,COUNTY SHERIFF,2014-12-03,2014-12-04,SUR,5000.0,1,NaN,NaN,0,1
10703,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,14DV00270,...,CITY,2014-03-04,2014-03-05,PR,1500.0,1,NaN,NaN,0,1


In [35]:
test['1yr'] = test.groupby('dedupe_id')['booking_date'].diff()

In [39]:
test['stayed'] = test['release_date'] - test['booking_date']

In [41]:
test['stayed'].head()

20701     0 days
20702   337 days
20703    17 days
20704     1 days
10703     1 days
Name: stayed, dtype: timedelta64[ns]

In [42]:
test['1yr'].head()

20701        NaT
20702   138 days
20703   894 days
20704   215 days
10703        NaT
Name: 1yr, dtype: timedelta64[ns]

In [46]:
test[['mni_no', '1yr', 'stayed']].head()

,mni_no,1yr,stayed
20701,494625,NaT,0 days
20702,494625,138 days,337 days
20703,494625,894 days,17 days
20704,494625,215 days,1 days
10703,673675,NaT,1 days


In [47]:
test.stayed = test.stayed.shift(1)

In [48]:
test[['mni_no', '1yr', 'stayed']].head()

,mni_no,1yr,stayed
20701,494625,NaT,NaT
20702,494625,138 days,0 days
20703,494625,894 days,337 days
20704,494625,215 days,17 days
10703,673675,NaT,1 days


In [54]:
test = test[test['1yr'].notnull()]

In [56]:
test[['mni_no', '1yr', 'stayed']].head()

,mni_no,1yr,stayed
20702,494625,138 days,0 days
20703,494625,894 days,337 days
20704,494625,215 days,17 days
10704,673675,387 days,1 days
10705,673675,335 days,1 days


In [57]:
test['after_released'] = test['1yr']-test['stayed']

In [58]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,1yr,stayed,after_released
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,0 SUR\ndtype: object,2500.0,0,NaN,NaN,0,1,138 days,0 days,138 days
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,SUR,1500.0,1,NaN,NaN,0,1,894 days,337 days,557 days
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,SUR,5000.0,1,NaN,NaN,0,1,215 days,17 days,198 days
10704,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,15DV00382,...,SUR,1000.0,1,NaN,NaN,0,1,387 days,1 days,386 days
10705,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,16DV00234,...,0 SUR\ndtype: object,2500.0,0,NaN,NaN,0,1,335 days,1 days,334 days
